## This is the algorithm used by the agent to learn; specifically a Double DQN Network using the neural network in [NN_Model.py](http://localhost:8888/notebooks/Deep_Reinforcement_Learning/Navigation-project/NN_Model.ipynb)

In [1]:
#Getting Libraries

import random
from collections import namedtuple, deque

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

#Getting function approximator

from NN_Model import QNetwork

In [2]:
# Assign variables

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 32  # minibatch size
GAMMA = 0.95  # discount factor
TAU = 5e-2  # for soft update of target parameters
LR = 5e-4  # learning rate
UPDATE_EVERY = 4  # how often to update the network
start_epsilon = 1.0  # initial epsilon value
epsilon_min = 0.1  # minimum epsilon value
epsilon_decay = 1000  # decay rate for exp decay per episode / decay time in episodes for linear decay

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#Defining the Agent class

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed=0):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """

        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.adaptive_eps = Adaptive_Epsilon(start_epsilon, epsilon_min, epsilon_decay)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed=seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed=seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)
        
        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
        
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, greedy=False):
        """Returns actions for given state for the current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if greedy or random.random() > self.adaptive_eps.get():
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        states.to(device)
        next_states.to(device)

        self.qnetwork_target.eval()
        self.qnetwork_local.eval()
        with torch.no_grad():
            best_actions = self.qnetwork_local(next_states).argmax(1).unsqueeze(1)
            q_targets = self.qnetwork_target(next_states).gather(1, best_actions)

        self.qnetwork_local.train()
        self.qnetwork_target.train()

        targets = rewards + (gamma * q_targets * (1 - dones))
        estimates = self.qnetwork_local(states).gather(1, actions)
        loss = F.mse_loss(estimates, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)


In [4]:
# Class for the epsilon-enabled updates

class Adaptive_Epsilon():
    """Class for adaptive epsilon with different decay types."""
    def __init__(self, eps_init, eps_min, decay_param, decay_type='linear'):
        update_functions = {'linear': self.linear_update, 'exp': self.exp_update}
        self.decay_type = decay_type if decay_type in update_functions.keys() else 'linear'
        self.update = update_functions[self.decay_type]
        self.eps_init = eps_init
        self.eps_min = eps_min
        self.decay_param = decay_param
        self.epsilon = self.eps_init
        self.lin_step = (self.eps_init - self.eps_min) / self.decay_param

    def get(self):
        return self.epsilon

    def linear_update(self):
        self.epsilon = max(self.epsilon - self.lin_step, self.eps_min)
        return self.epsilon

    def exp_update(self):
        self.epsilon = max(self.epsilon * self.decay_param, self.eps_min)
        return self.epsilon

    def reset(self):
        self.epsilon = self.eps_init

In [5]:
# Replay buffer in which the lessons from the episodes are stored

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(
            device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(
            device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)